In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from smogn import smoter
import smogn


In [ ]:
df = pd.read_csv('annonimized.csv')

In [ ]:
df.head()

In [ ]:
df.rename(columns={
    "concat('it001',`assignment_id`)": 'assignment_id',
    "concat('it001',`problem_id`)": 'problem_id',
    "concat('it001', username)": 'username',
    "concat('it001',`language_id`)": 'language_id'
}, inplace=True)

In [ ]:
df.head()

In [ ]:
df_tl = pd.read_csv('th-public.csv')

In [ ]:
df_tl.rename(columns={
    "hash": 'username'
}, inplace=True)

# Data Engineer

In [ ]:
score_df = df_tl[df_tl['username'].isin(df['username'])]

In [ ]:
score_df['TH'] = pd.to_numeric(df_tl['TH'], errors='coerce')

In [ ]:
print(score_df.dtypes)

In [ ]:
total_missing = df.isnull().sum().sum()

print("Total number of missing values in the DataFrame:")
print(total_missing)

In [ ]:
def create_try(score_df, df):
    try_count = df.groupby('username').size().reset_index(name='try')
    score_df = pd.merge(score_df, try_count, on='username', how='left')
    return score_df

In [ ]:
score_df = create_try(score_df, df)

In [ ]:
def create_solved(score_df, df):
    # Select relevant columns
    user_final_df = df[['username', 'is_final', 'pre_score']]
    
    # Filter rows where 'is_final' > 0 and 'pre_score' == 10000
    solved_df = user_final_df[(user_final_df['is_final'] > 0) & (user_final_df['pre_score'] == 10000)]
    
    # Group by 'username' and sum the 'is_final' column
    solved_df = solved_df.groupby('username').agg({'is_final': 'sum'})
    
    # Rename the 'is_final' column to 'solved'
    solved_df.rename(columns={'is_final':'solved'}, inplace=True)
    
    # Reset index to make 'username' a column again
    solved_df.reset_index(inplace=True)
    
    # Merge with the original score_df
    score_df = score_df.merge(solved_df, on='username', how='left')
    
    # Fill NaN values with 0.0
    # score_df.fillna(0.0, inplace=True)
    
    return score_df

In [ ]:
score_df = create_solved(score_df, df)

In [ ]:
def create_num_problem_submit(score_df, df):
    filtered_df = df[df['is_final'] == 1]
    unique_assignments = filtered_df.groupby('username')['problem_id'].nunique().reset_index(name='num_problems_submited')
    score_df = score_df.merge(unique_assignments, on='username', how='left')
    score_df['try'] = score_df['try'].fillna(0).astype(int)
    return score_df


In [ ]:
score_df = create_num_problem_submit(score_df, df)

In [ ]:
def create_num_assignment_submit(score_df, df):
    filtered_df = df[df['is_final'] == 1]
    unique_assignments = filtered_df.groupby('username')['assignment_id'].nunique().reset_index(name='num_assignments_submited')
    score_df = score_df.merge(unique_assignments, on='username', how='left')
    score_df['try'] = score_df['try'].fillna(0).astype(int)
    return score_df


In [ ]:
score_df = create_num_assignment_submit(score_df, df)

In [ ]:
'''
def create_total_score(score_df, df):
    user_score_df = df[['username', 'is_final', 'pre_score']]
    score = user_score_df[user_score_df['is_final'] > 0].groupby('username').sum()
    score = score.drop('is_final', axis=1)
    score.rename(columns={'pre_score': 'sum_score'}, inplace=True)
    score_df = pd.merge(score_df, score, on='username', how='left')
    return score_df
'''
def create_total_score(score_df, df):
    # Select relevant columns from df
    user_score_df = df[['username', 'is_final', 'pre_score', 'coefficient']]
    
    # Apply the coefficient to pre_score
    user_score_df['adjusted_score'] = user_score_df['pre_score'] / 100 * user_score_df['coefficient']
    
    # Filter out rows where is_final is less than or equal to 0
    filtered_score_df = user_score_df[user_score_df['is_final'] > 0]
    
    # Group by username and sum the adjusted scores
    score = filtered_score_df.groupby('username')['adjusted_score'].sum().reset_index()
    
    # Rename the column as sum_score
    score.rename(columns={'adjusted_score': 'sum_score'}, inplace=True)
    
    # Merge the summed scores back into the original score_df
    score_df = pd.merge(score_df, score, on='username', how='left')
    
    return score_df

In [ ]:
score_df = create_total_score(score_df, df)

In [ ]:
def create_score_per_problem(score_df):
    score_df['score_per_problem'] = score_df['sum_score'] / score_df['num_problems_submited']
    return score_df

In [ ]:
create_score_per_problem(score_df)

In [ ]:
score_df

In [ ]:
def apply_log_transformation(df, column, shift_value=1):
    """
    Apply a log transformation to a specified column in a DataFrame.
    If the column contains zero or negative values, shift the data by shift_value.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column (str): The name of the column to transform.
    shift_value (float): The value to add to the column to shift the data.

    Returns:
    pd.DataFrame: A new DataFrame with the log-transformed column.
    """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' does not exist in the DataFrame.")
    
    # Shift the data if there are zero or negative values
    if (df[column] <= 0).any():
        df_transformed = df.copy()
        df_transformed[column] = np.log(df_transformed[column] + shift_value)
        print(f"Data shifted by adding {shift_value} to handle zero or negative values.")
    else:
        df_transformed = df.copy()
        df_transformed[column] = np.log(df_transformed[column])
    
    return df_transformed

In [ ]:
##score_df = apply_log_transformation(score_df, 'TH')
##score_df = apply_log_transformation(score_df, 'try')
##score_df = apply_log_transformation(score_df, 'sum_score')

In [ ]:
numeric_cols = score_df.select_dtypes(include='number').columns

plt.figure(figsize=(12, 8))

for col in numeric_cols:
    sns.kdeplot(score_df[col], label=col)
    plt.title('KDE Plot for All Numeric Columns')
    plt.legend()
    plt.show()

In [ ]:
grouped_df = score_df.groupby('TH')['sum_score'].mean().reset_index()

# Plotting the data
plt.figure(figsize=(10, 6))
plt.scatter(grouped_df['TH'], grouped_df['sum_score'], color='blue', label='Mean of sum_score')
plt.xlabel('TH')
plt.ylabel('Mean of sum_score')
plt.title('TH vs Mean of sum_score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
total_missing = score_df.isnull().sum().sum()

print("Total number of missing values in the DataFrame:")
print(total_missing)

In [ ]:
fig= plt.figure(figsize = (40,30))
gs1 = fig.add_gridspec(3,3)
gs1.update(wspace=0.4, hspace=0.4)

axs0 = fig.add_subplot(gs1[0,0])
axs1 = fig.add_subplot(gs1[0,1])
axs2 = fig.add_subplot(gs1[1,0])

axes=[axs0,axs1,axs2]
background_color = '#f6f5f7'

for i in axes:
    i.set_facecolor(background_color)
fig.patch.set_facecolor(background_color) 
sns.scatterplot(ax = axs0, x = score_df['try'], y = score_df['TH'].sort_values() )
axs0.set_title('Solved Assignments and Grade Final')

fig.patch.set_facecolor(background_color) 
sns.scatterplot(ax = axs1, x = score_df['solved'], y = score_df['TH'].sort_values())
axs1.set_title('Solved/Tries Assignments and Grade Final')

fig.patch.set_facecolor(background_color) 
sns.scatterplot(ax = axs2,x = score_df['sum_score'], y = score_df['TH'].sort_values())
axs2.set_title('Submitted Assignments and Grade Final')

plt.show()

In [ ]:
grouped_df = df.drop_duplicates(subset=['username', 'assignment_id'])

# Step 2: Count the frequency of each assignment_id
assignment_counts = grouped_df['assignment_id'].value_counts().reset_index()
assignment_counts.columns = ['assignment_id', 'count']

# Step 3: Sort the counts in descending order
sorted_assignment_counts = assignment_counts.sort_values(by='count', ascending=False)

# Display the result
print("Sorted assignment counts:")
print(sorted_assignment_counts)

In [ ]:
corr = score_df.select_dtypes(include='number').corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

## Test

In [ ]:
test_df = score_df.copy()

In [ ]:
test_df = test_df.drop(columns=['username'])

In [ ]:
test_df = test_df.dropna()

In [ ]:
test_df = test_df[~((test_df['sum_score'] > 600000) & (test_df['sum_score'] < 700000) & (test_df['TH'] > 4) & (test_df['TH'] < 6))]
test_df = test_df[~((test_df['sum_score'] > 700000) & (test_df['TH'] < 6))]
test_df = test_df[~((test_df['sum_score'] < 300000) & (test_df['TH'] > 9))]

In [ ]:
numeric_cols = score_df.select_dtypes(include='number').columns

for col in numeric_cols:
    if col == 'TH':
        continue
    grouped_df = test_df.groupby('TH')[col].mean().reset_index()

    # Plotting the data
    plt.figure(figsize=(10, 6))
    plt.scatter(grouped_df['TH'], grouped_df[col], color='blue', label='Mean of sum_score')
    plt.xlabel('TH')
    plt.ylabel(f'Mean of ' + col)
    plt.title('TH vs Mean of ' + col)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
test_df

In [ ]:
print(test_df.isnull().sum())

In [ ]:
X = test_df.drop(columns=['TH'])
y = test_df['TH']

In [ ]:
import xgboost as xgb

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
test_df

In [ ]:
test_df.columns

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

##X = oversampled_df.drop(columns=['TH'])
##y = oversampled_df['TH']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

poly = PolynomialFeatures(degree=4)

X_train = poly.fit_transform(X_train)

X_test = poly.transform(X_test)

params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eta': 0.01,
    'eval_metric': 'rmse'
}

# Prepare the dataset and train the model
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)

evals = [(train_data, 'train'), (test_data, 'eval')]
model = xgb.train(params, train_data, num_boost_round=100, evals=evals, early_stopping_rounds=10)

y_pred = model.predict(test_data)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")

xgb.plot_importance(model)
plt.show()

residuals = y_test - y_pred

# Plot Actual vs. Predicted Values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', label='Predicted vs Actual')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Ideal Fit')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')
plt.legend()
plt.grid(True)
plt.show()

# Step 5: Plot Residuals
plt.figure(figsize=(10, 6))
sns.residplot(x=y_pred, y=residuals, lowess=True, line_kws={'color': 'red', 'lw': 2})
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(0, linestyle='--', color='gray')
plt.show()

# Assuming df and df_tl are defined somewhere in your code
predict_df = df[~df['username'].isin(df_tl['username'])]
unique_usernames_df = predict_df[['username']].drop_duplicates().reset_index(drop=True)

def create_predict_csv(predict_df, df, model):
    """
    Process predict_df, predict TBTL, and return a DataFrame with MSSV and TBTL.
    
    Parameters:
        predict_df (pd.DataFrame): DataFrame containing usernames to predict.
        df (pd.DataFrame): Original DataFrame used for aggregation and calculations.
        model: Trained model for predicting TBTL.

    Returns:
        pd.DataFrame: DataFrame containing 'MSSV' and predicted 'TBTL'.
    """
    # Step 1: Copy and process raw data
    raw = predict_df.copy()
    raw = create_try(raw, df)
    raw = create_solved(raw, df)
    raw = create_num_problem_submit(raw, df)
    raw = create_num_assignment_submit(raw, df)
    raw = create_total_score(raw, df)
    raw = create_score_per_problem(raw)

    # Step 2: Fill NaN values in numeric columns with column medians
    numeric_cols = raw.select_dtypes(include=['number']).columns
    raw[numeric_cols] = raw[numeric_cols].fillna(raw[numeric_cols].median())

    # Step 3: Prepare data for prediction
    X_predict = raw.drop(columns=['username'])  # Exclude 'username' from features
    usernames = raw['username']  # Save usernames for the final output

    X_predict = poly.transform(X_predict)

    # Step 4: Predict TBTL using the trained model
    predict_data = xgb.DMatrix(X_predict)  # Create DMatrix for prediction
    TBTL_predictions = model.predict(predict_data)

    # Step 5: Create a DataFrame with MSSV and predicted TBTL, rounded to 2 decimal places
    results_df = pd.DataFrame({
        'username': usernames,
        'TBTL': TBTL_predictions
    })
    results_df['TBTL'] = results_df['TBTL'].round(2)  # Round TBTL to 2 decimal places

    return results_df

result = create_predict_csv(unique_usernames_df, df, model)

result.to_csv('result_tbtl.csv', index=False, header=False)

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lasso', LassoCV(alphas=[0.001, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.1, 1, 10], cv=12, random_state=42))
])


In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
best_alpha = pipeline.named_steps['lasso'].alpha_
print(f"Best alpha: {best_alpha}")

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
r2 = r2_score(y_test, y_pred)
print(f"R^2 (Coefficient of Determination): {r2}")

In [ ]:
residuals = y_test - y_pred

# Step 5: Plot Residuals
plt.figure(figsize=(10, 6))
sns.residplot(x=y_pred, y=residuals, lowess=True, line_kws={'color': 'red', 'lw': 2})
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.axhline(0, linestyle='--', color='gray')
plt.show()

In [ ]:
predict_df = df[~df['username'].isin(df_tl['username'])]

In [ ]:
unique_usernames_df = predict_df[['username']].drop_duplicates().reset_index(drop=True)


In [ ]:
def create_predict_csv(predict_df, df, model):
    """
    Process predict_df, predict TBTL, and return a DataFrame with MSSV and TBTL.
    
    Parameters:
        predict_df (pd.DataFrame): DataFrame containing usernames to predict.
        df (pd.DataFrame): Original DataFrame used for aggregation and calculations.
        model: Trained model for predicting TBTL.

    Returns:
        pd.DataFrame: DataFrame containing 'MSSV' and predicted 'TBTL'.
    """
    # Step 1: Copy and process raw data
    raw = predict_df.copy()
    raw = create_try(raw, df)
    raw = create_solved(raw, df)
    raw = create_num_problem_submit(raw, df)
    raw = create_total_score(raw, df)
    ##raw = create_try_per_solve(raw)
    ##raw = create_submit_per_solve(raw)
    ##raw = apply_log_transformation(raw, 'try')
    ##raw = apply_log_transformation(raw, 'sum_score')

    ##raw = calculate_solved_per_try(raw)

    # Step 2: Fill NaN values in numeric columns with column medians
    numeric_cols = raw.select_dtypes(include=['number']).columns
    raw[numeric_cols] = raw[numeric_cols].fillna(raw[numeric_cols].median())

    # Step 3: Prepare data for prediction
    X_predict = raw.drop(columns=['username'])  # Exclude 'username' from features
    usernames = raw['username']  # Save usernames for the final output

    # Step 4: Predict TBTL using the trained model
    TBTL_predictions = model.predict(X_predict)

    # Step 5: Create a DataFrame with MSSV and predicted TBTL, rounded to 2 decimal places
    results_df = pd.DataFrame({
        'username': usernames,
        'TBTL': TBTL_predictions
    })
    results_df['TBTL'] = results_df['TBTL'].round(2)  # Round TBTL to 2 decimal places

    return results_df

In [ ]:
result = create_predict_csv(unique_usernames_df, df, pipeline)

In [ ]:
result.to_csv('result_tbtl.csv', index=False, header=False)